<a href="https://colab.research.google.com/github/eel-eel-eel/ric1340/blob/main/ch05_04_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境構築

Google Driveをマウント
（データセットや学習済みモデルを格納する）

パスワードを求められた場合はリンクをクリックし、Googleアカウントにログインして表示された文字列を入力する。

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

# 使用デバイスにGPUを設定
# 以下のような出力が出ていれば正常に設定ができている
# device(type='cuda', index=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## モデルの定義

In [ ]:
!pip install transformers[ja]==4.21.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 29.9 MB/s 
     |████████████████████████████████| 6.6 MB 66.8 MB/s 
     |████████████████████████████████| 182 kB 81.0 MB/s 
     |████████████████████████████████| 13.4 MB 58.0 MB/s 
     |████████████████████████████████| 47.4 MB 1.2 MB/s 
     |████████████████████████████████| 615 kB 72.8 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=9691b63440aa361dea95077bc7b7e7d7b5fccaf25b6c41a02726ac2a57ed1c03
  Stored in directory: /root/.cache/pip/wheels/45/b7/f5/a21e68db846eedcd00d69e37d60bab3f68eb20b1d99cdff652
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7425 sha256=f3ea904b1f7797e767c0faa7d474f88a4e30b90548a7a7429350ba91cb6e761a
  Stored in directory: /root/.cache/pip/wheels/22/bc/bb/46aba36d0388f67dfe44bb0edc20a2c964560d4d19ec394e05
  Created wheel for unidic-lite: fil

In [ ]:
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking",
    )

In [ ]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained(
    'cl-tohoku/bert-base-japanese-whole-word-masking'
    )

# 割り当てGPUによっては搭載GPUのメモリが少なくメモリエラーが発生することがあります。
# 画面上部のメニューから
# ランタイム -> ランタイムのタイプを変更 -> CPU -> 最初のセルを実行
# ランタイム -> ランタイムのタイプを変更 -> GPU
# を実施することで、別のGPUが割り当てられ解決する場合があります。
model.to(device)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model che

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

## DDQAデータセットでの学習

データセットのダウンロード

In [ ]:
!mkdir -p /content/drive/MyDrive/bert/5_3_ddqa

In [ ]:
cd /content/drive/MyDrive/bert/5_3_ddqa

/content/drive/MyDrive/bert/5_3_ddqa


本検証に用いるデータセットは利用条件に同意して手動でダウンロードする必要があります。  
下記の「ダウンロードページ」から`DDQA-1.0.tar.gz`をダウンロードして、上で作成したフォルダにアップロードしてください。
https://nlp.ist.i.kyoto-u.ac.jp/?Driving+domain+QA+datasets

In [ ]:
!tar -zxvf DDQA-1.0.tar.gz

DDQA-1.0/
DDQA-1.0/RC-QA/
DDQA-1.0/PAS-QA-NOM/
DDQA-1.0/PAS-QA-ACC/
DDQA-1.0/PAS-QA-DAT/
DDQA-1.0/README_en.txt
DDQA-1.0/README_ja.txt
DDQA-1.0/PAS-QA-DAT/DDQA-1.0_PAS-QA-DAT_train.json
DDQA-1.0/PAS-QA-DAT/DDQA-1.0_PAS-QA-DAT_dev.json
DDQA-1.0/PAS-QA-DAT/DDQA-1.0_PAS-QA-DAT_test.json
DDQA-1.0/PAS-QA-ACC/DDQA-1.0_PAS-QA-ACC_train.json
DDQA-1.0/PAS-QA-ACC/DDQA-1.0_PAS-QA-ACC_dev.json
DDQA-1.0/PAS-QA-ACC/DDQA-1.0_PAS-QA-ACC_test.json
DDQA-1.0/PAS-QA-NOM/DDQA-1.0_PAS-QA-NOM_dev.json
DDQA-1.0/PAS-QA-NOM/DDQA-1.0_PAS-QA-NOM_test.json
DDQA-1.0/PAS-QA-NOM/DDQA-1.0_PAS-QA-NOM_train.json
DDQA-1.0/RC-QA/DDQA-1.0_RC-QA_dev.json
DDQA-1.0/RC-QA/DDQA-1.0_RC-QA_test.json
DDQA-1.0/RC-QA/DDQA-1.0_RC-QA_train.json


## データセットの前処理

In [ ]:
import os, json
dataset_dir = "DDQA-1.0/RC-QA"
list_file = ["DDQA-1.0_RC-QA_train.json", "DDQA-1.0_RC-QA_dev.json", "DDQA-1.0_RC-QA_test.json"]
list_dataset = []

for fil in list_file:
  with open(os.path.join(dataset_dir, fil)) as f:
    dataset = json.load(f)
    list_dataset.append(dataset['data'][0]['paragraphs'])
    print(len(dataset['data'][0]['paragraphs']))

list_train, list_valid, list_test = list_dataset

8968
1053
1042


In [ ]:
# cl-tohoku/bert-base-japanese-whole-word-maskingのモデルは最大512トークンまで対応しているが、
# 学習時のGPUメモリ消費を抑えるため256としている
n_token = 256

In [ ]:
def is_in_span(idx, span):
  return span[0] <= idx and idx < span[1]

In [ ]:
from collections import defaultdict

def preprocess(examples, is_test=False):
  dataset = defaultdict(list)
  all_starts, all_ends = [], []

  for example in examples:
    for qa in example["qas"]:
      context, question, answers = example["context"], qa["question"], qa["answers"]
      starts, ends = [], []

      for i,answer in enumerate(answers):
        encode = tokenizer(question, context)["input_ids"]
        tokenized = tokenizer.decode(encode)

        decode_str = tokenized.replace(" ", "").replace("[CLS]", "").replace("[PAD]", "").replace("##", "")

        # decode後のコンテクストの開始位置（質問文長）
        len_question = decode_str.find('[SEP]')

        cnt = 0
        start_position = 0
        for i_t,e in enumerate(encode):
          tok = tokenizer.decode(e).replace(" ", "")

          if tok == "[CLS]" or tok == "[SEP]" or tok == "[PAD]":
            continue
          else:
            if cnt <= len_question + answer["answer_start"]:
              start_position = i_t
            if cnt <= len_question + answer["answer_start"] + len(answer["text"]):
              end_position = i_t

          cnt += len(tok.replace("##", ""))

        starts.append(start_position)
        ends.append(end_position)

        if (not is_test) or (i == 0):
          dataset["contexts"].append(context)
          dataset["questions"].append(question)
          dataset["input_ids"].append(encode)
          dataset["tokenized"].append(tokenized)

          dataset["start_positions"].append(start_position)
          dataset["end_positions"].append(end_position)

      all_starts.append(starts)
      all_ends.append(ends)
  all_answers = (all_starts, all_ends)
  return dataset, all_answers

In [ ]:
# preprocessの例
ex1 = {"context":"警察にもお願いがあります。高速道は飛ばすところであります。それよりも危険運転を厳しく取り締まってもらいたい。栃木県警、埼玉県警、福島県警、何のためにヘリコプターを装備しているのですか。",
      "qas":[{"id":"56958372310021_00",
              "question":"誰にお願いがあるか？",
              "answers":[{"text":"警察", "answer_start":0}],
              "is_impossible":False}]}
print(preprocess([ex1]))

(defaultdict(<class 'list'>, {'contexts': ['警察にもお願いがあります。高速道は飛ばすところであります。それよりも危険運転を厳しく取り締まってもらいたい。栃木県警、埼玉県警、福島県警、何のためにヘリコプターを装備しているのですか。'], 'questions': ['誰にお願いがあるか？'], 'input_ids': [[2, 3654, 7, 24050, 14, 31, 29, 2935, 3, 1573, 7, 28, 24050, 14, 130, 2610, 8, 1942, 405, 9, 787, 12222, 1134, 12, 130, 2610, 8, 218, 221, 28, 3164, 1498, 11, 9047, 19657, 628, 16, 11633, 1549, 8, 5857, 16582, 6, 3205, 16582, 6, 3191, 16582, 6, 1037, 5, 82, 7, 8213, 11, 2124, 15, 16, 33, 5, 2992, 29, 8, 3]], 'tokenized': ['[CLS] 誰 に お願い が ある か? [SEP] 警察 に も お願い が あり ます 。 高速 道 は 飛ばす ところ で あり ます 。 それ より も 危険 運転 を 厳しく 取り締まっ て もらい たい 。 栃木 県警 、 埼玉 県警 、 福島 県警 、 何 の ため に ヘリコプター を 装備 し て いる の です か 。 [SEP]'], 'start_positions': [9], 'end_positions': [10]}), ([[9]], [[10]]))


In [ ]:
for k,v in ex1.items():
  print(k,v)
print('---')
for k,v in preprocess([ex1])[0].items():
  print(k,v)

context 警察にもお願いがあります。高速道は飛ばすところであります。それよりも危険運転を厳しく取り締まってもらいたい。栃木県警、埼玉県警、福島県警、何のためにヘリコプターを装備しているのですか。
qas [{'id': '56958372310021_00', 'question': '誰にお願いがあるか？', 'answers': [{'text': '警察', 'answer_start': 0}], 'is_impossible': False}]
---
contexts ['警察にもお願いがあります。高速道は飛ばすところであります。それよりも危険運転を厳しく取り締まってもらいたい。栃木県警、埼玉県警、福島県警、何のためにヘリコプターを装備しているのですか。']
questions ['誰にお願いがあるか？']
input_ids [[2, 3654, 7, 24050, 14, 31, 29, 2935, 3, 1573, 7, 28, 24050, 14, 130, 2610, 8, 1942, 405, 9, 787, 12222, 1134, 12, 130, 2610, 8, 218, 221, 28, 3164, 1498, 11, 9047, 19657, 628, 16, 11633, 1549, 8, 5857, 16582, 6, 3205, 16582, 6, 3191, 16582, 6, 1037, 5, 82, 7, 8213, 11, 2124, 15, 16, 33, 5, 2992, 29, 8, 3]]
tokenized ['[CLS] 誰 に お願い が ある か? [SEP] 警察 に も お願い が あり ます 。 高速 道 は 飛ばす ところ で あり ます 。 それ より も 危険 運転 を 厳しく 取り締まっ て もらい たい 。 栃木 県警 、 埼玉 県警 、 福島 県警 、 何 の ため に ヘリコプター を 装備 し て いる の です か 。 [SEP]']
start_positions [9]
end_positions [10]


In [ ]:
# preprocessの例2
ex2 = {"context":"またまた小西タンの九州スイッチのＣＭに出てくる某県の話。以下は友達の友達が体験した話。指○スカイラインという有料道路がある。昼間は有料だが夜１０時なると無料になる道路だ。",
       "qas":[{"id":"57017617480007_00",
               "question":"どの道路が昼間は有料だが夜１０時なると無料になりますか？",
               "answers":[{"text":"指○スカイライン", "answer_start":43},
                          {"text":"指○スカイラインという有料道路", "answer_start":43}],
               "is_impossible":False}]}
print(preprocess([ex2]))

(defaultdict(<class 'list'>, {'contexts': ['またまた小西タンの九州スイッチのＣＭに出てくる某県の話。以下は友達の友達が体験した話。指○スカイラインという有料道路がある。昼間は有料だが夜１０時なると無料になる道路だ。', 'またまた小西タンの九州スイッチのＣＭに出てくる某県の話。以下は友達の友達が体験した話。指○スカイラインという有料道路がある。昼間は有料だが夜１０時なると無料になる道路だ。'], 'questions': ['どの道路が昼間は有料だが夜１０時なると無料になりますか？', 'どの道路が昼間は有料だが夜１０時なると無料になりますか？'], 'input_ids': [[2, 3219, 1305, 14, 11995, 9, 7700, 75, 14, 1563, 121, 72, 139, 13, 4691, 7, 297, 2610, 29, 2935, 3, 106, 28491, 28447, 19523, 2797, 5, 2446, 9017, 5, 2968, 7, 71, 16, 2501, 20201, 149, 5, 735, 8, 562, 9, 12455, 5, 12455, 14, 4946, 15, 10, 735, 8, 254, 4478, 23354, 140, 7700, 1305, 14, 31, 8, 11995, 9, 7700, 75, 14, 1563, 121, 72, 139, 13, 4691, 7, 139, 1305, 75, 8, 3], [2, 3219, 1305, 14, 11995, 9, 7700, 75, 14, 1563, 121, 72, 139, 13, 4691, 7, 297, 2610, 29, 2935, 3, 106, 28491, 28447, 19523, 2797, 5, 2446, 9017, 5, 2968, 7, 71, 16, 2501, 20201, 149, 5, 735, 8, 562, 9, 12455, 5, 12455, 14, 4946, 15, 10, 735, 8, 254, 4478, 23354, 140, 7700, 1305, 14, 31, 8, 11995, 9, 7700, 75

In [ ]:
for k,v in ex2.items():
  print(k,v)
print('---')
for k,v in preprocess([ex2])[0].items():
  print(k,v)

context またまた小西タンの九州スイッチのＣＭに出てくる某県の話。以下は友達の友達が体験した話。指○スカイラインという有料道路がある。昼間は有料だが夜１０時なると無料になる道路だ。
qas [{'id': '57017617480007_00', 'question': 'どの道路が昼間は有料だが夜１０時なると無料になりますか？', 'answers': [{'text': '指○スカイライン', 'answer_start': 43}, {'text': '指○スカイラインという有料道路', 'answer_start': 43}], 'is_impossible': False}]
---
contexts ['またまた小西タンの九州スイッチのＣＭに出てくる某県の話。以下は友達の友達が体験した話。指○スカイラインという有料道路がある。昼間は有料だが夜１０時なると無料になる道路だ。', 'またまた小西タンの九州スイッチのＣＭに出てくる某県の話。以下は友達の友達が体験した話。指○スカイラインという有料道路がある。昼間は有料だが夜１０時なると無料になる道路だ。']
questions ['どの道路が昼間は有料だが夜１０時なると無料になりますか？', 'どの道路が昼間は有料だが夜１０時なると無料になりますか？']
input_ids [[2, 3219, 1305, 14, 11995, 9, 7700, 75, 14, 1563, 121, 72, 139, 13, 4691, 7, 297, 2610, 29, 2935, 3, 106, 28491, 28447, 19523, 2797, 5, 2446, 9017, 5, 2968, 7, 71, 16, 2501, 20201, 149, 5, 735, 8, 562, 9, 12455, 5, 12455, 14, 4946, 15, 10, 735, 8, 254, 4478, 23354, 140, 7700, 1305, 14, 31, 8, 11995, 9, 7700, 75, 14, 1563, 121, 72, 139, 13, 4691, 7, 139, 1305, 75, 8, 3], [2, 3219, 1305, 14, 11995, 9, 7700, 75, 14, 1563,

In [ ]:
from torch.utils.data import Dataset, DataLoader

class QADataset(Dataset):
  def __init__(self, dataset, is_test=False):
    self.dataset = dataset
    self.is_test = is_test

  def __getitem__(self, idx):
    data = {'input_ids': torch.tensor(self.dataset["input_ids"][idx], device=device)}
    if not self.is_test:
      data["start_positions"] = torch.tensor(self.dataset["start_positions"][idx], device=device)
      data["end_positions"] = torch.tensor(self.dataset["end_positions"][idx], device=device)
    return data

  def __len__(self):
    return len(self.dataset["input_ids"])

In [ ]:
pp_test, test_answers = preprocess(list_test, is_test=True)

In [ ]:
dataset_train = QADataset(preprocess(list_train)[0])
dataset_valid = QADataset(preprocess(list_valid)[0])
pp_test, test_answers = preprocess(list_test, is_test=True)
dataset_test = QADataset(pp_test, is_test=True)

In [ ]:
len(dataset_train), len(dataset_valid), len(dataset_test)

(25195, 1542, 1045)

##質問応答モデルの作成

In [ ]:
from transformers import Trainer, TrainingArguments
training_config = TrainingArguments(
  output_dir = './results',
  num_train_epochs = 1,
  per_device_train_batch_size = 8,
  per_device_eval_batch_size = 8,
  warmup_steps = 500,
  weight_decay = 0.1,
  do_eval = True,
  save_steps = 5000
)

trainer = Trainer(
    model = model,
    args = training_config,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    eval_dataset = dataset_valid
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25195
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3150


Step,Training Loss
500,2.679600
1000,1.620800
1500,1.456800
2000,1.335400
2500,1.197100
3000,1.148000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3150, training_loss=1.5498384360661583, metrics={'train_runtime': 913.1601, 'train_samples_per_second': 27.591, 'train_steps_per_second': 3.45, 'total_flos': 2210045927092176.0, 'train_loss': 1.5498384360661583, 'epoch': 1.0})

## モデルの評価

In [ ]:
result = trainer.predict(dataset_test)

***** Running Prediction *****
  Num examples = 1045
  Batch size = 8


In [ ]:
import numpy as np
predictions = (np.argmax(result[0][0], axis=1), np.argmax(result[0][1], axis=1))

In [ ]:
# トークン単位でのExact Match（厳密一致）とF1を計算

def evaluate(ground_truth, predictions):
  em, f1 = 0., 0.
  n_data = len(ground_truth[0])
  for answer_starts, answer_ends, pred_start, pred_end in zip(ground_truth[0], ground_truth[1], predictions[0], predictions[1]):
    for answer_start, answer_end in zip(answer_starts, answer_ends):
      if pred_start == answer_start and pred_end == answer_end:
        em += 1
        break

    f1_candidate = [calc_f1(ps, pe, pred_start, pred_end) for ps, pe in zip(answer_starts, answer_ends)]
    f1 += max(f1_candidate)
  return {"em": (em / n_data), "f1": (f1 / n_data)}

def calc_f1(gt_start, gt_end, pred_start, pred_end):
  tp = max(0, (1 + min(gt_end, pred_end) - max(gt_start, pred_start)))
  precision = tp / (1 + pred_end - pred_start)  if 1 + pred_end - pred_start > 0 else 0
  # 通常、1 + gt_end - gt_start > 0がFalseになることはあり得ないが念のため
  recall = tp / (1 + gt_end - gt_start) if 1 + gt_end - gt_start > 0 else 0
  if precision * recall > 0:
    return 2 * (precision * recall) / (precision + recall)
  return 0.

In [ ]:
evaluate(test_answers, predictions)

{'em': 0.7626794258373206, 'f1': 0.8723363048588432}